In [1]:
!pip install pandas

In [6]:
import pandas as pd
df = pd.read_csv(r"C:\Users\ASUS\Desktop\projectforecastpm2_5\dataforecast\raw_data\export-jsps001-1d.csv")  # เปลี่ยนเป็นชื่อไฟล์จริง
df['timestamp'] = pd.to_datetime(df['timestamp'])
df = df.set_index("timestamp")
df.drop(columns=['timezone'], inplace=True)
df.drop(columns=['pm_2_5_sp'], inplace=True)
df.drop(columns=['Unnamed: 0'], inplace=True)
df.drop(columns=['humidity'], inplace=True)
df.drop(columns=['temperature'], inplace=True)
# df.isna().sum()
for i in range(1, 8):
    df[f'pm_2_5_Lag{i}'] = df['pm_2_5'].shift(i)
df.dropna(inplace=True)
#,timestamp,timezone,humidity,pm_2_5,pm_2_5_sp,temperature
df.to_csv(r"C:\Users\ASUS\Desktop\projectforecastpm2_5\dataforecast\cleandata_v2_day.csv", index=False)
df

,pm_2_5,pm_2_5_Lag1,pm_2_5_Lag2,pm_2_5_Lag3,pm_2_5_Lag4,pm_2_5_Lag5,pm_2_5_Lag6,pm_2_5_Lag7
timestamp,,,,,,,,
2021-08-22 07:00:00,11.972368,11.279905,9.832737,7.994888,15.163043,13.293243,19.432905,29.242424
2021-08-23 07:00:00,7.357038,11.972368,11.279905,9.832737,7.994888,15.163043,13.293243,19.432905
2021-08-24 07:00:00,7.196071,7.357038,11.972368,11.279905,9.832737,7.994888,15.163043,13.293243
2021-08-25 07:00:00,9.834989,7.196071,7.357038,11.972368,11.279905,9.832737,7.994888,15.163043
2021-08-26 07:00:00,7.374415,9.834989,7.196071,7.357038,11.972368,11.279905,9.832737,7.994888
...,...,...,...,...,...,...,...,...
2025-02-08 07:00:00,33.923883,27.864365,28.153336,29.342907,33.424542,53.791623,43.738720,39.954672
2025-02-09 07:00:00,32.647832,33.923883,27.864365,28.153336,29.342907,33.424542,53.791623,43.738720
2025-02-10 07:00:00,32.856690,32.647832,33.923883,27.864365,28.153336,29.342907,33.424542,53.791623


In [1]:
import pandas as pd
df = pd.read_csv(r"C:\Users\ASUS\Desktop\projectforecastpm2_5\dataforecast\raw_data\export-jsps001-1h.csv")  # เปลี่ยนเป็นชื่อไฟล์จริง
df['timestamp'] = pd.to_datetime(df['timestamp'])
df.drop(columns=['timezone'], inplace=True)
df.drop(columns=['pm_2_5_sp'], inplace=True)
df.drop(columns=['Unnamed: 0'], inplace=True)

# df.isna().sum()
for i in range(1, 8): # สร้าง Lag ของค่า PM2.5 ในช่วง 7 ชั่วโมงก่อนหน้า
    df[f'pm_2_5_Lag{i}'] = df['pm_2_5'].shift(i)

# df['PM2.5_rolling_mean'] = df['pm_2_5'].rolling(window=3).mean()
# df['rolling_avg_7'] = df['pm_2_5'].rolling(window=2).mean()

# target_days = 7
# hours_per_day = 24

# for day in range(1, target_days + 1):
#     lag_hours = day * hours_per_day
#     df[f'pm_2_5_Lag{day}day'] = df['pm_2_5'].shift(lag_hours)

# ลบแถวที่มีค่า NaN จากการสร้าง Lag
df.dropna(inplace=True)

# df.drop(columns=['pm_2_5_Lag1'], inplace=True)


df.to_csv(r"C:\Users\ASUS\Desktop\projectforecastpm2_5\dataforecast\cleandata_vtest_hours.csv", index=False)
df

,timestamp,humidity,pm_2_5,temperature,pm_2_5_Lag1,pm_2_5_Lag2,pm_2_5_Lag3,pm_2_5_Lag4,pm_2_5_Lag5,pm_2_5_Lag6,pm_2_5_Lag7
7,2021-08-15 14:00:00,75.818644,20.084746,29.238136,20.285714,43.163793,55.907563,34.826923,19.655172,25.800000,29.242424
8,2021-08-15 15:00:00,74.075424,22.758621,29.374576,20.084746,20.285714,43.163793,55.907563,34.826923,19.655172,25.800000
9,2021-08-15 16:00:00,79.509167,18.478261,28.073333,22.758621,20.084746,20.285714,43.163793,55.907563,34.826923,19.655172
10,2021-08-15 17:00:00,90.900000,8.225000,25.560000,18.478261,22.758621,20.084746,20.285714,43.163793,55.907563,34.826923
11,2021-08-15 18:00:00,90.565833,6.193277,25.547500,8.225000,18.478261,22.758621,20.084746,20.285714,43.163793,55.907563
...,...,...,...,...,...,...,...,...,...,...,...
27544,2025-02-11 08:00:00,87.340026,43.263158,23.013561,42.558333,37.806723,34.691667,35.283333,38.275000,36.193277,34.616667
27545,2025-02-11 09:00:00,80.816273,42.478992,25.590026,43.263158,42.558333,37.806723,34.691667,35.283333,38.275000,36.193277
27546,2025-02-11 10:00:00,67.041454,40.848214,29.023077,42.478992,43.263158,42.558333,37.806723,34.691667,35.283333,38.275000
27547,2025-02-11 11:00:00,53.353909,31.842105,32.314087,40.848214,42.478992,43.263158,42.558333,37.806723,34.691667,35.283333


In [17]:
#check
x = df.iloc[4135]
x


timestamp      2022-05-03 22:00:00
humidity                     100.0
pm_2_5                    5.033333
temperature              26.686167
pm_2_5_Lag1               4.883333
pm_2_5_Lag2                   9.45
pm_2_5_Lag3               5.233333
pm_2_5_Lag4               2.216667
pm_2_5_Lag5               3.283333
pm_2_5_Lag6              16.357143
pm_2_5_Lag7              12.216667
Name: 4142, dtype: object